Скрипт генерации датасета для тестирования модели игры.
Структура конечного файла:

- word - имя существительное в именительном падеже (могут повторяться, если для слова есть более одного определения)

- defs - словарное определение слова

In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [56]:
file_name = 'wikitionary_dataset/wikitionary_dataset.csv'

data = pd.read_csv(file_name, delimiter='\\')

print(data.shape)
data.head()

(785566, 5)


,Заглавная страница,Unnamed: 1,[],"{'synonyms': [], 'hyponyms': [], 'hypernyms': [], 'antonyms': []}",[].1
0,Фемиксира,noun,[('согласно легендам греческой мифологии — гор...,"{'synonyms': ['', ''], 'hyponyms': ['', ''], '...",['']
1,эбонитовый,other,"[('произведённый из эбонита', ['Для этого опыт...","{'synonyms': ['-', 'чёрный'], 'hyponyms': ['',...",['']
2,а,noun,[('название первой буквы русского алфавита или...,"{'synonyms': ['и : аз'], 'hyponyms': ['—', '']...","['от а до я (от а до зет)', 'сказал а, говори ..."
3,Ба,noun,"[('демон засухи в древнекитайской мифологии', ...","{'synonyms': ['', ''], 'hyponyms': ['', ''], '...",['']
4,в,other,"[('внутрь чего-либо (предмета или места)', ['П...","{'synonyms': ['внутрь', 'част. на', 'через', '...",['типовые сочетания с предлогом в']


In [57]:
df = data.copy()

# renaming columns
df.rename(columns={
    df.columns[0]: 'word',
    df.columns[1]: 'POS',
    df.columns[2]: 'defs_str',
    df.columns[3]: 'rels_str',
    df.columns[4]: 'phras_str'
}, inplace = True)

df.shape

(785566, 5)

Модифицируем вид данных.  
Оставим только существительные.

In [58]:
df_nouns = df[df['POS'].map(lambda x: x == 'noun')]

print(df_nouns.shape)
df_nouns.head()

(254278, 5)


,word,POS,defs_str,rels_str,phras_str
0,Фемиксира,noun,[('согласно легендам греческой мифологии — гор...,"{'synonyms': ['', ''], 'hyponyms': ['', ''], '...",['']
2,а,noun,[('название первой буквы русского алфавита или...,"{'synonyms': ['и : аз'], 'hyponyms': ['—', '']...","['от а до я (от а до зет)', 'сказал а, говори ..."
3,Ба,noun,"[('демон засухи в древнекитайской мифологии', ...","{'synonyms': ['', ''], 'hyponyms': ['', ''], '...",['']
6,ахинея,noun,"[('вздор, бессмыслица', ['Пока мы не будем счи...","{'synonyms': ['чушь', 'вздор', 'бессмыслица', ...","['нести ахинею / пороть ахинею', '']"
7,агат,noun,"[('полудрагоценный поликристаллический камень,...","{'synonyms': ['—', 'уголь', 'ночь', '—'], 'hyp...",['']


В датасете много нерусских слов (татарских, тувинских, ингушских и др.). Используем ещё один датасет - словарный список русских слов, для фильтрации.
Загрузим список слов:

In [59]:
russian = set()
with open('russian-words/russian.txt', encoding='cp1251') as f:
    for line in f:
        russian.add(line[:-1])

Оставим только слова из словаря:

In [60]:
df_nouns = df_nouns[df_nouns['word'].map(lambda x: x in russian)]

print(df_nouns.shape)
df_nouns.head()

(62175, 5)


,word,POS,defs_str,rels_str,phras_str
2,а,noun,[('название первой буквы русского алфавита или...,"{'synonyms': ['и : аз'], 'hyponyms': ['—', '']...","['от а до я (от а до зет)', 'сказал а, говори ..."
6,ахинея,noun,"[('вздор, бессмыслица', ['Пока мы не будем счи...","{'synonyms': ['чушь', 'вздор', 'бессмыслица', ...","['нести ахинею / пороть ахинею', '']"
7,агат,noun,"[('полудрагоценный поликристаллический камень,...","{'synonyms': ['—', 'уголь', 'ночь', '—'], 'hyp...",['']
8,Новосибирск,noun,"[('город в России, центр Новосибирской области...","{'synonyms': ['сибирская столица', 'столица Си...",[]
9,мальчик,noun,"[('ребёнок либо подросток мужского пола', ['Ма...","{'synonyms': ['частичные: малец', 'мальчуган',...","['а был ли мальчик?', 'борьба нанайских мальчи..."


Удалим все слова, содержащие символы кроме кириллицы в нижнем регистре и дефиса (в том числе имена собственные).

In [61]:
import re

word_pattern = re.compile('^[а-я-]+$')
df_nouns = df_nouns[df_nouns['word'].map(lambda x: bool(re.match(word_pattern, str(x))))]

print(df_nouns.shape)
df_nouns.head()

(56911, 5)


,word,POS,defs_str,rels_str,phras_str
2,а,noun,[('название первой буквы русского алфавита или...,"{'synonyms': ['и : аз'], 'hyponyms': ['—', '']...","['от а до я (от а до зет)', 'сказал а, говори ..."
6,ахинея,noun,"[('вздор, бессмыслица', ['Пока мы не будем счи...","{'synonyms': ['чушь', 'вздор', 'бессмыслица', ...","['нести ахинею / пороть ахинею', '']"
7,агат,noun,"[('полудрагоценный поликристаллический камень,...","{'synonyms': ['—', 'уголь', 'ночь', '—'], 'hyp...",['']
9,мальчик,noun,"[('ребёнок либо подросток мужского пола', ['Ма...","{'synonyms': ['частичные: малец', 'мальчуган',...","['а был ли мальчик?', 'борьба нанайских мальчи..."
10,публицист,noun,"[('журналист, пишущий о политике и общественно...","{'synonyms': [''], 'hyponyms': [''], 'hypernym...",[]


Определения прочитаем в соответствии с их структурой (как списки), лишние колонки удалим.

In [62]:
defs = df_nouns['defs_str'].map(lambda x: eval(x))

df_lists = pd.DataFrame({'defs': defs})
df_nouns = pd.concat([df_nouns, df_lists], axis = 1)

df_nouns = df_nouns.drop(['POS', 'defs_str', 'rels_str', 'phras_str'], axis=1)

Упорядочим по алфавиту.

In [63]:
df_nouns = df_nouns.sort_values(by=['word'])

Отфильтруем только содержательные определения, склеим датасет слово - одно определение.

In [64]:
words = []
defs = []

def_pattern = re.compile('^[А-Яа-я- .,:;()]+$')

for row in range(df_nouns.shape[0]):
    for i in df_nouns.iloc[row]['defs']:
        definition = i[0]
        if definition != '' and re.match(def_pattern, definition):
            words.append(df_nouns.iloc[row]['word'])
            defs.append(definition)

test_set = pd.DataFrame({'word': words, 'defs': defs})

Проверим итоговый вид:

In [65]:
print(test_set.shape)
test_set

(50544, 2)


,defs,word
0,название первой буквы русского алфавита или зв...,а
1,"нечто первое в ряду аналогичных предметов, пун...",а
2,отец,аб
3,вид грубого домотканого сукна,аба
4,грубая шерстяная материя белого цвета,аба
5,"часть светильника, колпак (чаще всего в виде п...",абажур
6,устройство для крепления абажура осветительног...,абажуродержатель
7,", персидская и старая грузинская серебряная мо...",абаз
8,"черноморский суровый восточный ветер, дующий о...",абаза
9,бестолковый басурманин,абаза


Сохраним test_set:

In [66]:
test_set.to_csv('rus_test_set.csv',index=False)